
# Implementing 3 layered radial fiber using Bragg exact

The two layer case works, so the transfer matrix maybe is having the problems

In [ ]:
import numpy as np

from fiberamp.fiber.microstruct.bragg import BraggExact
from step_exact import RadialStep  # to check work on bragg exact
from step_exact import plotlogf
from ngsolve.webgui import Draw
from ngsolve import CF
from scipy.optimize import newton
from scipy.linalg import null_space

In [ ]:
n_air = 1.00027717
n_glass = 1.4388164768221814

In [ ]:
A = BraggExact(ts=[15e-6,.5*15e-6, 15e-6], mats=['air', 'glass', 'air'], 
               ns=[lambda x:n_air, lambda x:n_glass, lambda x:n_air],
              maxhs=[.2,.03, .1], wl=3.6e-6)


In [ ]:
B = RadialStep(r1=15e-6, t1=.5*15e-6, t2=15e-6, scale=15e-6,
                 n1=1.00027717, n2=1.4388164768221814, n3=1.00027717,
                 R1_maxh=.1, R2_maxh=.1, R3_maxh=.1, wavelength=3.6e-6)

In [ ]:
k_low = A.k0 * A.ns[0] * A.scale
k_low

In [ ]:
k_low = B.k0 * B.n1 * B.scale
k_low

In [ ]:
outer = 'h2'
nu = 1

In [ ]:
plotlogf(A.determinant, .97*k_low, 1.001*k_low, -.1,.1, nu, outer,
         iref=100, rref=100, levels=100)

In [ ]:
plotlogf(B.hybrid_det_matrix, .97*k_low, 1.001*k_low, -.1,.1, nu, outer,
         iref=100, rref=100, levels=100, loop=True)

In [ ]:
guess = np.array(.997 * k_low)

beta1 = newton(A.determinant, guess, args=(nu, outer), tol = 1e-15)

print("Scaled beta: ", beta1, ". Residual of determinant: ", abs(A.determinant(beta1, nu, outer)))


In [ ]:
guess = np.array(26.06)

beta2 = newton(B.hybrid_det_matrix, guess, args=(nu, outer), tol = 1e-15)

print("Scaled beta: ", beta2, ". Residual of determinant: ", abs(B.hybrid_det_matrix(beta2, nu, outer)))


# Coefficients

Got the determinant right after fixing transfer matrix, but coefficients/field building was still broken.  Check coefficients

In [ ]:
A1, B1, C, D, a, b, c, d, \
    alpha, beta = A.determinant(beta1, nu, outer, return_coeffs=True)

In [ ]:
v1, v2 = B1, -A1
v = np.array([v1, v2])

w1, w2 = (a*v1 + b*v2) / alpha, (c*v1 + d*v2) / beta1

rhos = A.rhos
ns = A.ns

M = np.zeros((len(A.rhos), 4), dtype=complex)
L = np.zeros((4, 2), dtype=complex)

L[:, :] = np.eye(4)[:, [0, 2]]
L = L @ v
L

In [ ]:
M[0, :] = L

M

In [ ]:
for i in range(len(rhos)-2):
    nl, nr = ns[i], ns[i+1]
    rho = rhos[i]
    L = A.transfer_matrix(beta1, nu, rho, nl, nr) @ L
    M[i+1, :] = L



In [ ]:
M

In [ ]:
if outer == 'h2':
    inds = [1, 3]
elif outer == 'h1':
    inds = [0, 2]
else:
    raise TypeError("Outer function must be 'h1' (guided) or 'h2'\
(leaky).")

M[-1, inds] = w1, w2

In [ ]:
M

# Found issue

We overwrote our beta propagation constant with a coefficient we also called beta in the code.  Now it works

# Bragg Exact Fields

In [ ]:
Fs = A.all_fields(beta1, nu, outer)

In [ ]:
Draw(1e1*Fs['Ez'], A.mesh)

In [ ]:
Draw(Fs['Ephi'], A.mesh)

In [ ]:
Draw(1e1*Fs['Hz'], A.mesh)

In [ ]:
Draw(Fs['Hphi'], A.mesh)

# RadialStep Exact Fields

We know this is correct.  Didn't vectorize or evaluate coefficients from explicit determinant yet so this often fails unless loss is high for mode under consideration

In [ ]:
B.hybrid_coefficients_matrix(beta2, nu, outer)

In [ ]:
Fs = B.hybrid_fields_matrix(beta2, nu, outer)

In [ ]:
Draw(1e1*Fs['Ez'], A.mesh)

In [ ]:
Draw(1e-1*Fs['Ephi'], A.mesh)